In [1]:
!head /home/hadoop/dataset/penumpang-10-2021.csv

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col

In [3]:
spark = SparkSession.builder.appName('DataFrame Basics').getOrCreate()

23/02/15 14:10:14 WARN Utils: Your hostname, dl247-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.15.130 instead (on interface ens33)
23/02/15 14:10:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/15 14:10:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.csv('/home/hadoop/dataset/penumpang-10-2021.csv', header=True)

In [5]:
df_group = df.groupBy('jenis', 'bulan').agg(avg('jumlah_penumpang'))

In [6]:
df_group.filter(df_group.jenis=='BRT').show()

+-----+-----+---------------------+
|jenis|bulan|avg(jumlah_penumpang)|
+-----+-----+---------------------+
|  BRT|   10|    399719.8461538461|
+-----+-----+---------------------+



In [7]:
df09 = spark.read.csv('/home/hadoop/dataset/penumpang-09-2021.csv', header=True)
df10 = spark.read.csv('/home/hadoop/dataset/penumpang-10-2021.csv', header=True)
df11 = spark.read.csv('/home/hadoop/dataset/penumpang-11-2021.csv', header=True)
df12 = spark.read.csv('/home/hadoop/dataset/penumpang-12-2021.csv', header=True)

In [8]:
from functools import reduce

dflist = [df09, df10, df11, df12]

# create merged dataframe
df = reduce(pyspark.sql.DataFrame.unionAll, dflist)

In [9]:
#df.describe().show()

In [20]:
df.show()

+-----+-----+----------+-----------+--------------------+----------------+
|tahun|bulan|     jenis|kode_trayek|              trayek|jumlah_penumpang|
+-----+-----+----------+-----------+--------------------+----------------+
| 2021|   09|Mikrotrans|     JAK.88|Terminal Tanjung ...|           21569|
| 2021|   09|Mikrotrans|     JAK.85|Bintara - Cipinan...|           23109|
| 2021|   09|Mikrotrans|     JAK.84|Terminal Kampung ...|           36842|
| 2021|   09|Mikrotrans|     JAK.80|Rawa Buaya - Rawa...|           46648|
| 2021|   09|Mikrotrans|      JA.77|Tanjung Priok - J...|           48302|
| 2021|   09|Mikrotrans|     JAK.75|Cililitan - Kp. Pulo|           35365|
| 2021|   09|Mikrotrans|     JAK.74|Terminal Rawamang...|           30072|
| 2021|   09|Mikrotrans|     JAK.73|Jambore Cibubur -...|           59539|
| 2021|   09|Mikrotrans|     JAK.72|Kampung Rambutan ...|           64779|
| 2021|   09|Mikrotrans|     JAK.71|Kampung Rambutan ...|           48561|
| 2021|   09|Mikrotrans| 

In [10]:
df.select('jenis').distinct().show(truncate=False)

+-----------------------+
|jenis                  |
+-----------------------+
|BRT                    |
|Mikrotrans             |
|Angkutan Umum Integrasi|
+-----------------------+



In [11]:
ref = spark.createDataFrame([('BRT', 1),
                                ('Mikrotrans', 2),
                                ('Angkutan Umum Integrasi', 3),
                                ('Lain-lain', 4)], ["type", "kode"])

In [19]:
ref.show(truncate=False)

+-----------------------+----+
|type                   |kode|
+-----------------------+----+
|BRT                    |1   |
|Mikrotrans             |2   |
|Angkutan Umum Integrasi|3   |
|Lain-lain              |4   |
+-----------------------+----+



In [12]:
df_joined = df.join(ref,df['jenis'] == ref['type'])

In [13]:
df_cast = df_joined.withColumn('jumlah_penumpang',col('jumlah_penumpang').cast("int"))

In [14]:
df_agg = df_cast.groupBy('jenis', 'bulan').agg(avg('jumlah_penumpang'))

In [15]:
df_filter = df_agg.filter(df_agg.jenis=='BRT')

In [21]:
df_filter.show()

+-----+-----+---------------------+
|jenis|bulan|avg(jumlah_penumpang)|
+-----+-----+---------------------+
|  BRT|   11|   461483.76923076925|
|  BRT|   12|   465652.07692307694|
|  BRT|   10|    399719.8461538461|
|  BRT|   09|   336068.53846153844|
+-----+-----+---------------------+



In [22]:
df_filter.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[jenis#84, bulan#83], functions=[avg(jumlah_penumpang#229)])
   +- Exchange hashpartitioning(jenis#84, bulan#83, 200), ENSURE_REQUIREMENTS, [plan_id=487]
      +- HashAggregate(keys=[jenis#84, bulan#83], functions=[partial_avg(jumlah_penumpang#229)])
         +- Project [bulan#83, jenis#84, cast(jumlah_penumpang#87 as int) AS jumlah_penumpang#229]
            +- BroadcastHashJoin [jenis#84], [type#209], Inner, BuildLeft, false
               :- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, false]),false), [plan_id=482]
               :  +- Union
               :     :- Filter (isnotnull(jenis#84) AND (jenis#84 = BRT))
               :     :  +- FileScan csv [bulan#83,jenis#84,jumlah_penumpang#87] Batched: false, DataFilters: [isnotnull(jenis#84), (jenis#84 = BRT)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/hadoop/dataset/penumpang-09-2021.csv], PartitionFilters: [], Pu